 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/NLP%20for%20predicting%20success.ipynb#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="http://localhost:8888/notebooks/NLP%20for%20predicting%20success.ipynb#Import-Data" data-toc-modified-id="Import-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import Data</a></span></li><li><span><a href="http://localhost:8888/notebooks/NLP%20for%20predicting%20success.ipynb#NLP-on-the-Storyline" data-toc-modified-id="NLP-on-the-Storyline-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>NLP on the Storyline</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/NLP%20for%20predicting%20success.ipynb#Data-Preparation" data-toc-modified-id="Data-Preparation-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Data Preparation</a></span></li><li><span><a href="http://localhost:8888/notebooks/NLP%20for%20predicting%20success.ipynb#Cleaning-the-storylines" data-toc-modified-id="Cleaning-the-storylines-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Cleaning the storylines</a></span></li><li><span><a href="http://localhost:8888/notebooks/NLP%20for%20predicting%20success.ipynb#Analysis-of-Success" data-toc-modified-id="Analysis-of-Success-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Analysis of Success</a></span></li><li><span><a href="http://localhost:8888/notebooks/NLP%20for%20predicting%20success.ipynb#Cross-Validation" data-toc-modified-id="Cross-Validation-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Cross Validation</a></span></li><li><span><a href="http://localhost:8888/notebooks/NLP%20for%20predicting%20success.ipynb#Analysis" data-toc-modified-id="Analysis-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Analysis</a></span></li></ul></li></ul></div>

# Import Libraries

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [95]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import ParameterGrid

In [37]:
import bs4 as bs
import nltk
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

eng_stopwords = stopwords.words('english')

# Import Data

In [2]:
data = pd.read_csv('data_ready.csv', sep=';', encoding='utf-8')

In [3]:
data.head()

,Actor_1_id,Actor_1_name,Actor_2_id,Actor_2_name,Actor_3_id,Actor_3_name,Country,Director_id,Director_name,Genres,...,isWestern,Duration_movie_final,Bugdet_final,Gross_final,Opening Weekend final,Actor_1_like_final,Actor_2_like_final,Actor_3_like_final,Director_like_final,IMDb_rating_final
0,nm0941777,Sam Worthington,nm0757855,Zoe Saldana,nm0000244,Sigourney Weaver,UK,nm0000116,\nJames Cameron,"[' Action', ' Adventure', ' Fantasy', ' Sci-Fi']",...,0,162.0,237000000.0,2.787965e+09,77025481.0,5300.0,6.0,2.0,7000.0,7.8
1,nm3766090,Doug Walker,nm3597657,Rob Walker,NaN,NaN,NaN,nm3766090,\nDoug Walker,[' Documentary'],...,0,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,6.1
2,nm0000136,Johnny Depp,nm0089217,Orlando Bloom,nm0461136,Keira Knightley,USA,nm0893659,\nGore Verbinski,"[' Action', ' Adventure', ' Fantasy']",...,0,169.0,300000000.0,9.634204e+08,139802190.0,40000.0,5100.0,7700.0,582.0,7.1
3,nm0185819,Daniel Craig,nm0910607,Christoph Waltz,nm2244205,Léa Seydoux,UK,nm0005222,\nSam Mendes,"[' Action', ' Adventure', ' Thriller']",...,0,148.0,245000000.0,8.806742e+08,70403148.0,8500.0,11000.0,1800.0,0.0,6.8
4,nm0000288,Christian Bale,nm0362766,Tom Hardy,nm0004266,Anne Hathaway,UK,nm0634240,\nChristopher Nolan,"[' Action', ' Thriller']",...,0,164.0,250000000.0,1.084939e+09,160887295.0,3.0,3.0,11000.0,2.0,8.4


In [4]:
data.isnull().sum()

Actor_1_id                 25
Actor_1_name               25
Actor_2_id                 38
Actor_2_name               38
Actor_3_id                 46
Actor_3_name               46
Country                    38
Director_id               134
Director_name             134
Genres                      0
IMDb_critics               53
Keywords                    0
Language                   46
Movie_name                  0
Storyline                  42
Year                      156
label_country              40
label_language             48
isAction                    0
isAdventure                 0
isAnimation                 0
isBiography                 0
isComedy                    0
isCrime                     0
isFantasy                   0
isDocumentary               0
isFamily                    0
isDrama                     0
isHistory                   0
isHorror                    0
isMusic                     0
isMystery                   0
isRomance                   0
isSciFi   

# NLP on the Storyline

## Data Preparation

In [15]:
index_gross = pd.isnull(data['Gross_final'])
X = data['Storyline'][~index_gross]
y = data[['Gross_final','Bugdet_final']][~index_gross]

In [18]:
y_success  = np.where(y['Gross_final'] >= 2 * y['Bugdet_final'], 1, 0)

In [20]:
y_success[0:10]

array([1, 1, 1, 1, 0, 0, 1, 1, 1, 1])

In [36]:
lengths = X.apply(len)

print('Average character length of the reviews are:')
print (np.mean(lengths))

Average character length of the reviews are:
578.523052464229


## Cleaning the storylines

In [41]:
X[0]

'\nWhen his brother is killed in a robbery, paraplegic Marine Jake Sully decides to take his place in a mission on the distant world of Pandora. There he learns of greedy corporate figurehead Parker Selfridge\'s intentions of driving off the native humanoid "Na\'vi" in order to mine for the precious material scattered throughout their rich woodland. In exchange for the spinal surgery that will fix his legs, Jake gathers intel for the cooperating military unit spearheaded by gung-ho Colonel Quaritch, while simultaneously attempting to infiltrate the Na\'vi people with the use of an "avatar" identity. While Jake begins to bond with the native tribe and quickly falls in love with the beautiful alien Neytiri, the restless Colonel moves forward with his ruthless extermination tactics, forcing the soldier to take a stand - and fight back in an epic battle for the fate of Pandora.                '

In [65]:
def review_cleaner(review):
    '''
    Clean and preprocess a review.
    
    1. Remove HTML tags
    2. Use regex to remove all special characters (only keep letters)
    3. Make strings to lower case and tokenize / word split reviews
    4. Remove English stopwords
    5. Rejoin to one string
    '''
    
    #1. Remove HTML tags
    review = bs.BeautifulSoup(review).text
    
    #2. Use regex to find emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', review)
    
    #3. Remove punctuation
    review = re.sub("[^a-zA-Z]", " ",review)
    
    #4. Tokenize into words (all lower case)
    review = review.lower().split()
    
    #5. Remove stopwords
    eng_stopwords = set(stopwords.words("english"))
    review = [w for w in review if not w in eng_stopwords]
    
    #6. Join the review to one sentence
    review = ' '.join(review+emoticons)
    # add emoticons to the end

    return(review)

In [66]:
review_cleaner(X[2])

/Users/Romain/.virtualenvs/romainvenv/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


'elizabeth captain barbossa rescue captain jack sparrow land dead must face foes davy jones lord cutler beckett beckett control jones heart forms dark alliance order rule seas wipe last pirates jack barbossa elizabeth tia dalma crew must call pirate lords four corners globe including infamous sao feng gathering pirate lords want release goddess calypso davy jones damned lover trap sent fear pirate lords must combine pieces bound ritual undo release hopes help fight pirates stand together make final stand freedom beckett jones norrington flying dutchman entire east india trading company'

In [67]:
num_story = len(X)

story_clean_original = []

for story in X:
    story_clean_original.append(review_cleaner(story))

/Users/Romain/.virtualenvs/romainvenv/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [68]:
# Porter stemming on the results in review_clean_original

story_clean_ps = []

ps = PorterStemmer()


for i in range(0,num_story): 
    ps_stems = []
    for w in story_clean_original[i].split():
        if w == 'oed':
            continue
        ps_stems.append(ps.stem(w))
    
    story_clean_ps.append(' '.join(ps_stems))

In [69]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

In [70]:
# Lemmatizer

story_clean_wnl = []

wnl = WordNetLemmatizer()

for i in range(0,num_story):
    wnl_stems = []
    token_tag = pos_tag(story_clean_original[i].split())
    for pair in token_tag:
        res = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))
        wnl_stems.append(res)

    story_clean_wnl.append(' '.join(wnl_stems))

## Analysis of Success

In [72]:
seed = 42
test_size = int(X.shape[0]/5.0)

X_train, X_test, y_train, y_test = train_test_split(X, y_success, test_size = test_size, random_state = seed)
print("train set size", X_train.shape)
print("test set size", X_test.shape)
print("Proportion of success in train set", sum(y_train)/float(y_train.shape[0]))
print("Proportion of success in test set", sum(y_test)/float(y_test.shape[0]))

train set size (1887,)
test set size (471,)
Proportion of success in train set 0.6444091149973503
Proportion of success in test set 0.6496815286624203


In [73]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000)

In [74]:
vectorizer.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [75]:
print(vectorizer.get_feature_names()[:10])

['00', '000', '007', '10', '100', '1000', '11', '12', '13', '14']


In [76]:
train_bag = vectorizer.transform(X_train) #transform to a feature matrix
test_bag = vectorizer.transform(X_test)

In [77]:
print(train_bag.toarray().shape)
print(test_bag.toarray().shape)

(1887, 5000)
(471, 5000)


In [78]:
from  sklearn.ensemble  import RandomForestClassifier

## Initialize a Random Forest classifier with 50 trees
# hyperparameter n_estimators always set in instantiation

forest = RandomForestClassifier(n_estimators = 50)

In [79]:
forest = forest.fit(train_bag, y_train)

In [80]:
train_predictions = forest.predict(train_bag)
valid_predictions = forest.predict(test_bag)

In [81]:
metrics.accuracy_score(y_train,train_predictions)

0.9994700582935877

In [82]:
metrics.accuracy_score(y_test,valid_predictions)

0.6666666666666666

In [83]:
metrics.confusion_matrix(y_test,valid_predictions)

array([[ 25, 140],
       [ 17, 289]])

In [84]:
tn, fp, fn, tp = metrics.confusion_matrix(y_test,valid_predictions).ravel()

In [85]:
importances = forest.feature_importances_
# returns relative importance of all features.
# they are in the order of the columns
print(importances)

[0.         0.00024067 0.         ... 0.         0.         0.        ]


In [86]:
# sort importance scores# sort i 
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
top_10 = indices[:10]

# Get top ten features
print([vectorizer.get_feature_names()[ind] for ind in top_10])

Feature ranking:
['and', 'the', 'to', 'brother', 'is', 'in', 'of', 'but', 'his', 'young']


## Cross Validation

In [92]:
params = {'max_features': [500, 2000, 5000, 7000], 
          'n_estimators': [10, 25 , 50, 75],
         'stop_words': [None, 'english']}

In [96]:
def cv(parameters):
    best_acc_score = 0
    index = 1
    best_grid = None
    
    for g in ParameterGrid(parameters):
        print('test number', index)
        index = index + 1
        max_features = g['max_features']
        n_estimators = g['n_estimators']
        stop_words = g['stop_words']

        vectorizer = CountVectorizer(analyzer = "word", preprocessor = None, stop_words = stop_words, max_features = max_features)
        vectorizer.fit(X_train)
        train_bag = vectorizer.transform(X_train) #transform to a feature matrix
        test_bag = vectorizer.transform(X_test)

        forest = RandomForestClassifier(n_estimators = n_estimators)
        forest = forest.fit(train_bag, y_train)
    
        train_predictions = forest.predict(train_bag)
        valid_predictions = forest.predict(test_bag)

        acc_score = metrics.accuracy_score(y_test,valid_predictions)
        
        print("Accuracy score: %0.5f" % acc_score)
        print("------------------------------------")
            
            #keep the best AUC score
        if acc_score > best_acc_score:
            best_acc_score = acc_score
            best_grid = g
        print("Best intermediate AUC score: %0.5f" % best_acc_score)
        print(" ")
    print("Final Best AUC score: %0.5f" % best_acc_score) 
    print("Grid:", best_grid)
    return best_grid

In [97]:
cv(params)

test number 1
Accuracy score: 0.56900
------------------------------------
Best intermediate AUC score: 0.56900
 
test number 2
Accuracy score: 0.60722
------------------------------------
Best intermediate AUC score: 0.60722
 
test number 3
Accuracy score: 0.63694
------------------------------------
Best intermediate AUC score: 0.63694
 
test number 4
Accuracy score: 0.63482
------------------------------------
Best intermediate AUC score: 0.63694
 
test number 5
Accuracy score: 0.65817
------------------------------------
Best intermediate AUC score: 0.65817
 
test number 6
Accuracy score: 0.62208
------------------------------------
Best intermediate AUC score: 0.65817
 
test number 7
Accuracy score: 0.66242
------------------------------------
Best intermediate AUC score: 0.66242
 
test number 8
Accuracy score: 0.63907
------------------------------------
Best intermediate AUC score: 0.66242
 
test number 9
Accuracy score: 0.63270
------------------------------------
Best intermed

{'max_features': 2000, 'n_estimators': 50, 'stop_words': None}

## Analysis

In [100]:
vectorizer = CountVectorizer(analyzer = "word", preprocessor = None, stop_words = None, max_features = 2000)
vectorizer.fit(X_train)
train_bag = vectorizer.transform(X_train) #transform to a feature matrix
test_bag = vectorizer.transform(X_test)

forest = RandomForestClassifier(n_estimators = 500)
forest = forest.fit(train_bag, y_train)
    
train_predictions = forest.predict(train_bag)
valid_predictions = forest.predict(test_bag)

acc_score = metrics.accuracy_score(y_test,valid_predictions)

In [101]:
print('the accuracy on the test set is:', acc_score)
metrics.confusion_matrix(y_test,valid_predictions).ravel()

the accuracy on the test set is: 0.6602972399150743


array([ 14, 151,   9, 297])

In [102]:
tn, fp, fn, tp = metrics.confusion_matrix(y_test,valid_predictions).ravel()

In [104]:
print('the TPR rate is:', tp / (fn + tp))
print('The FPR rate is:', fp / (fp + tn))

the TPR rate is: 0.9705882352941176
The FPR rate is: 0.9151515151515152
